In [381]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [382]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Configuration

In [383]:
image_rows, image_columns, image_depth = 64, 64, 96
batch_size = 2
epochs = 100
learning_rate = 0.01

Load your new dataset

In [384]:
data = []  # To store video data
labels = []  # To store corresponding labels

Modify this section to load your dataset

In [385]:
dataset_path = '/content/drive/MyDrive/VRN-Group-6/Dataset/me-cuts/cuts'  # Change this to the path of your dataset
excel_file_path = '/content/drive/MyDrive/VRN-Group-6/Dataset/me-cuts/Micros_dataset.xls'
class_folders = os.listdir(dataset_path)
class_labels = {}  # Define a mapping of class labels to integers
current_label = 0

# Read the Excel file into a DataFrame
data_frame = pd.read_excel(excel_file_path)

# Print the first few rows to verify the data
print(data_frame.head())


   Subject  ID  StartFrame  EndFrame                     VideoFile  \
0        1   1          52        65  european-poker-Barcelona.mp4   
1        2   1          92       100  european-poker-Barcelona.mp4   
2        2   2           7        35  european-poker-Barcelona.mp4   
3        3   1          82        90  european-poker-Barcelona.mp4   
4        4   1          18        29  european-poker-Barcelona.mp4   

   StartFrameVideo  EndFrameVideo  Length   AU   Emotion  \
0            64967          64980      13  L12  contempt   
1            11170          11172       2  R12  contempt   
2            45910          45938      28  NaN       NaN   
3            18683          18691       8  1.2  surprise   
4            23317          23328      11  1.2  surprise   

                               Note  
0                               NaN  
1                               NaN  
2  smile, but not a microexpression  
3                               NaN  
4  more macro-expression, oc

In [386]:
# Sample mapping between textual labels and numerical indices
emotion_mapping = {
    "contempt": 0,
    "smile": 1,
    "surprise": 2,
    "anger": 3,
    "fear": 4,
    # Add more labels as needed
}

data_frame["Label"] = data_frame["Emotion"].map(emotion_mapping)

# Replace NaN values with a default label (-1 in this example)
data_frame["Label"].fillna(-1, inplace=True)

# Convert labels to integers
data_frame["Label"] = data_frame["Label"].astype(int)

# Check unique labels after the conversion
unique_labels = data_frame["Label"].unique()

In [387]:
# Initialize data and labels lists
data = []
labels = []

# Iterate through the DataFrame
for index, row in data_frame.iterrows():
    subject = row['Subject']
    id = row['ID']
    start_frame = row['StartFrame']
    end_frame = row['EndFrame']
    emotion = row['Emotion']

    if not pd.isna(emotion):  # Skip rows without emotion labels
        # Check if the emotion label exists in the mapping; if not, assign a new label
        if emotion not in class_labels:
            class_labels[emotion] = len(class_labels)

        class_label = class_labels[emotion]

        video_file = f'sub{subject}-{id}.mp4'
        video_path = os.path.join(dataset_path, video_file)
        cap = cv2.VideoCapture(video_path)

        video_frames = []  # To store frames of the current video

        # Read frames within the specified range
        for frame_number in range(start_frame, end_frame + 1):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = cap.read()
            if not ret:
                break
            frame_resized = cv2.resize(frame, (image_columns, image_rows), interpolation=cv2.INTER_AREA)
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
            gray_frame = gray_frame.astype('float32')  # Convert to float32
            gray_frame -= np.mean(gray_frame)
            max_value = np.max(gray_frame)
            if max_value > 0:
                gray_frame /= max_value
            video_frames.append(gray_frame)

        cap.release()

        if len(video_frames) > 0:
            # Append data and labels only when valid frames are available
            data.append(video_frames)
            labels.append(class_label)


In [388]:
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))
# print(data)
# print(labels)


13
13


<ipython-input-388-ca3ee89e44c4>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


Convert labels to one-hot encoding

In [389]:
# labels = to_categorical(data_frame["Label"], num_classes=len(emotion_mapping))

Data preprocessing

In [390]:
# if data.size > 0:
#     data = data.astype('float32')
#     if np.max(data) > 0:
#         data -= np.mean(data)
#         data /= np.max(data)

Split data into training and validation sets

In [391]:
# Split data into training and validation sets
print(len(data))
print(len(labels))

max_frames = 78  # Set to a fixed maximum number of frames
reshaped_data = np.zeros((len(data), image_rows, image_columns, max_frames))

for i, video_frames in enumerate(data):
    num_frames = len(video_frames)
    frame_size = (image_columns, image_rows)  # Set the desired frame size

    # Create an array to store the resized frames
    resized_frames = np.zeros((num_frames, image_rows, image_columns))

    for j, frame in enumerate(video_frames):
        # Resize each frame to the desired size
        resized_frame = cv2.resize(frame, frame_size, interpolation=cv2.INTER_AREA)
        resized_frames[j, :, :] = resized_frame

    if num_frames <= max_frames:
        # If the number of frames is less than the maximum, pad the frames with zeros
        reshaped_data[i, :, :, :num_frames] = np.moveaxis(resized_frames, 0, -1)
    else:
        # If the number of frames exceeds the maximum, truncate the frames
        reshaped_data[i, :, :, :] = np.moveaxis(resized_frames[:max_frames], 0, -1)


num_classes = len(unique_labels)
labels = [label if 0 <= label < num_classes else num_classes - 1 for label in labels]

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=num_classes)

# # Now you can convert reshaped_data to TensorFlow tensors
# train_data = tf.convert_to_tensor(reshaped_data, dtype=tf.float32)
# train_labels = to_categorical(labels, num_classes=num_classes)


# train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Split data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

train_data = np.array(train_data)
train_labels = np.array(train_labels)
val_data = np.array(val_data)
val_labels = np.array(val_labels)


13
13


Model architecture

In [392]:
# Model architecture
model = Sequential()
model.add(Conv3D(32, (3, 3, 15), input_shape=(image_rows, image_columns, max_frames, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Compile the model

In [393]:
# Compile the model
sgd = SGD(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

Model summary

In [394]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_14 (Conv3D)          (None, 62, 62, 64, 32)    4352      
                                                                 
 max_pooling3d_14 (MaxPooli  (None, 20, 20, 21, 32)    0         
 ng3D)                                                           
                                                                 
 dropout_28 (Dropout)        (None, 20, 20, 21, 32)    0         
                                                                 
 flatten_14 (Flatten)        (None, 268800)            0         
                                                                 
 dense_28 (Dense)            (None, 128)               34406528  
                                                                 
 dropout_29 (Dropout)        (None, 128)               0         
                                                     

Model checkpoint

In [395]:
checkpoint = ModelCheckpoint('weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Training the model

In [400]:
def data_generator(data, labels, batch_size):
    num_samples = len(data)
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            if len(batch_indices) < batch_size:  # Skip incomplete batches
                continue
            batch_data = [data[i] for i in batch_indices]  # Select data for the batch
            batch_labels = [labels[i] for i in batch_indices]  # Select labels for the batch
            yield np.array(batch_data), np.array(batch_labels)

# Ensure the shape of train_data and val_data is (image_rows, image_columns, max_frames, 1)
train_data = np.expand_dims(train_data, axis=-1)
val_data = np.expand_dims(val_data, axis=-1)

# Calculate the number of steps for training and validation
train_steps = len(train_data) // batch_size
val_steps = len(val_data) // batch_size

# Train the model using the data generator
model.fit(data_generator(train_data, train_labels, batch_size), steps_per_epoch=train_steps, validation_data=data_generator(val_data, val_labels, batch_size), validation_steps=val_steps, epochs=epochs, callbacks=callbacks_list)

ValueError: ignored